In [29]:
%load_ext autoreload
%autoreload 2
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

#indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
indexed_circuit = [4,5,3,5,0,5,5,5,5,4,5,5,2]
#indexed_circuit = [5,3,3,5,4,0,5,5,3,3,5,3,5]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable)
print(model(cirq.resolve_parameters(circuit,resolver)), energy)


tf.Tensor([-7.9999995], shape=(1,), dtype=float32) tf.Tensor(-7.9999995, shape=(), dtype=float32)


In [31]:
#Mindexed_circuit, Mresolver, Mindex_to_symbols = iid_inserter.place_identities(indexed_circuit,symbol_to_value=resolver,rate_iids_per_step=2)
Mcircuit = vqe_handler.give_unitary(indexed_circuit, resolver)
Mcircuit

(0, 0): ───Rx(-0.5π)────────────────────────────@───Rx(2π)──────Rz(0.5π)───────────────────────────────────────────────────────────
                                                │
(0, 1): ───Rx(0)───────Rz(0.235π)───Rx(1.51π)───X───Rx(1.86π)───Rx(-0.212π)───Rx(-0.999π)───Rx(-1.53π)───Rx(1.36π)───Rx(-0.995π)───

In [32]:
i,r,po=simplifier.reduce_circuit(indexed_circuit,resolver,index_to_symbols)

7


In [33]:
nu=vqe_handler.give_unitary(i,r)
nu

(0, 0): ───Rx(-0.5π)────────────────────────@───Rx(2π)───Rz(0.5π)───
                                            │
(0, 1): ───Rx(0)───────Rz(0.235π)───Rx(π)───X───────────────────────

In [34]:
model(nu)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-8.], dtype=float32)>